### Author: Ally Sprik
### Last-updated: 25-02-2024

Goal of this notebook is to generate a full panel of possible minimal sets and evaluate them using ROC analysis.
NOTE: The code will be stopped before getting to teh full analysis, as it takes a long time to run.
There should be a CSV file with the results, at least for what I tested, but if you want to run it again, run the code from that part on.



In [ ]:
import pandas as pd
import numpy as np
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
from collections import Counter
import itertools

net = gum.loadBN("../3. Model/Fitted_Networks/R_WP_all_952.net")

df_brno = pd.read_csv("../0.1. Cleaned_data/Cleaned_Brno_model_complete.csv")

df_brno_LNM = df_brno.dropna(subset=["LNM"])
df_brno_Survival = df_brno.dropna(subset=["Survival5yr"])

brno_LNM_true = df_brno_LNM["LNM"]
brno_Surv_true = df_brno_Survival["Survival5yr"]


The following code is to generate all possible minimal sets of variables

In [ ]:
evidence_columns = ["ER", "PR", "p53", "L1CAM", "CA125", "Platelets", "Cytology", "MRI_MI", "MSI", "POLE", "PreoperativeGrade"]

sets = []
for i in range(1, len(evidence_columns)+1):
    els = [list(x) for x in itertools.combinations(evidence_columns, i)]
    sets.extend(els)

len(sets)

Defining the functions to get the probabilities

In [ ]:
def getProbabilities(model, evidence_LNM, evidence_Surv, Surv = "Survival5yr", samples = 100):
    ls_result_LNM = []
    ls_result_Surv = []
    
    #for i in range(1, samples):
    resultsLNM = []
    resultsSurvival = []
    net = gum.LazyPropagation(model)
    net.getNumberOfThreads()
    net.setNumberOfThreads(10)
    
    for j in range(len(evidence_LNM)):
        evidencerow = evidence_LNM.iloc[j]
        evidencerow = evidencerow.dropna().to_dict()
        
        try:
            net.setEvidence(evidencerow)
            
            net.makeInference()

            resultLNM = net.posterior("LNM")
            
            resultsLNM.append(resultLNM)
        except Exception as error:
            print("Error at row regarding LNM", j)
            print(error)
            
            resultsLNM.append(resultLNM)

    
    for j in range(len(evidence_Surv)):
        evidencerow = evidence_Surv.iloc[j]
        evidencerow = evidencerow.dropna().to_dict()
        
        try:
            net.setEvidence(evidencerow)
            
            net.makeInference()

            resultSurvival = net.posterior("Survival5yr")
            
            resultsSurvival.append(resultSurvival)
        except Exception as error:
            print("Error at row regarding Survival", j)
            print(error)
            
            resultsSurvival.append(resultSurvival)

        
    return resultsLNM, resultsSurvival

def getProbResults(results, target):
    res = []
    
    for i in range(len(results)):
        res.append(results[i][target])
    return pd.DataFrame(res)

Defining the functions to calculate mean and confidence intervals

In [ ]:
from scipy.stats import sem, t

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    
    m, se = np.mean(a), sem(a)
    h = se * t.ppf((1 + confidence) / 2., n-1)
    
    return m, m-h, m+h

The following code raise an error that not all the code will automatically run, as it takes a long time to run.

In [ ]:
# Throw error to prevent running the following code, 
# Only run if you truly want to recalculate
raise ValueError("This will take a long time to run, are you sure you want to run it? \n Otherwise see below for loading in the results and continuing from there")

The following code defines a function to take in a set and calculate the ROC for the LNM and Survival, using bootstrapping, finally returning the average ROC and the confidence intervals

In [ ]:
from sklearn.metrics import roc_auc_score
import multiprocessing

ROC_results = pd.DataFrame(columns=["Set", "Full Net LNM", "Full Net Survival"])


def getSetROC(set, net = net, df_brno_LNM = df_brno_LNM, df_brno_Survival = df_brno_Survival, brno_LNM_true = brno_LNM_true, brno_Surv_true = brno_Surv_true):
    
    bootstrap_samples = 1000
    samples = 100
    
    # Print what number of set is being processed
    print("Set number: ", sets.index(set)+1, " of ", len(sets))
    results_LNM, results_Survival = getProbabilities(net, df_brno_LNM[list(set)], df_brno_Survival[list(set)])
    
    LNM_prob = getProbResults(results_LNM, 1)
    Survival_prob = getProbResults(results_Survival, 1)
        
    LNM_ROC = []
    Survival_ROC = []
    
    skipped_rows = []
    
    for i in range(bootstrap_samples):
        LNM_prob_temp = LNM_prob.sample(n=samples, replace=True)
        Survival_prob_temp = Survival_prob.sample(n=samples, replace=True)

        LNM_res_temp = brno_LNM_true.iloc[LNM_prob_temp.index]
        Survival_res_temp = brno_Surv_true.iloc[Survival_prob_temp.index]
        try:
            LNM_ROC.append(roc_auc_score(LNM_res_temp, LNM_prob_temp))
            Survival_ROC.append(roc_auc_score(Survival_res_temp, Survival_prob_temp))
        except:
            #"Error in a bootstrap, skipping"
            skipped_rows.append(i)
            
    LNM_ROC = [x for i, x in enumerate(LNM_ROC) if i not in skipped_rows]
    Survival_ROC = [x for i, x in enumerate(Survival_ROC) if i not in skipped_rows]
    
    LNM_mean, LNM_lower, LNM_upper = mean_confidence_interval(LNM_ROC)
    Survival_mean, Survival_lower, Survival_upper = mean_confidence_interval(Survival_ROC)
        
    LNM = str(round(LNM_mean, 3)) + " [" + str(round(LNM_lower, 3)) + ", " + str(round(LNM_upper, 3)) + "]"
    Survival = str(round(Survival_mean, 3)) + " [" + str(round(Survival_lower, 3)) + ", " + str(round(Survival_upper, 3)) + "]"
    
    res = pd.DataFrame([[set, LNM, Survival]], columns=["Set", "Full Net LNM", "Full Net Survival"])
    
    return res
       

The following code uses multiprocessing to run the function on all the sets

In [ ]:
 Processes = (multiprocessing.cpu_count() - 2) # Set the number of cores on your system, use 1 if you don't want to use multiprocessing, (multiprocessing.cpu_count() - 1) to use maximum cores available (I would not recommend using without -1, as it will make your system unresponsive)
with multiprocessing.Pool(Processes) as pool:
    x =  pool.map(getSetROC, sets)
    
    for i in x:
        ROC_results = pd.concat([ROC_results, i], ignore_index=True)
    

The following code saves the results to a CSV file

In [ ]:
# Save this because you really don't want to run it again
ROC_results.to_csv("fullPanelMinimalSets.csv", index=False)

# If you only want to analyse use the following code
continue from here on if you want to analyse the results

In [ ]:
ROC_results = pd.read_csv("fullPanelMinimalSets.csv")

# Code to generate the average ROC for sets containing a certain variable
Created to gain an idea of the variables, and which ones are part of the best sets


In [ ]:
# LNM
# What is the Average ROC for sets containing certain variables, with the range of ROC values 
print("LNM")
# For CA125
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "CA125" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "CA125" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "CA125" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing CA125: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For MRI_MI
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "MRI_MI" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "MRI_MI" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "MRI_MI" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing MRI_MI: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For MSI
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "MSI" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "MSI" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "MSI" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing MSI: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For POLE
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "POLE" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "POLE" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "POLE" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing POLE: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For PreoperativeGrade
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "PreoperativeGrade" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "PreoperativeGrade" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "PreoperativeGrade" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing PreoperativeGrade: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For ER
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "ER" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "ER" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "ER" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing ER: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For PR
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "PR" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "PR" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "PR" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing PR: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For p53
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "p53" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "p53" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "p53" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing p53: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For L1CAM
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "L1CAM" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "L1CAM" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "L1CAM" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing L1CAM: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For Platelets
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "Platelets" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "Platelets" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "Platelets" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing Platelets: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For Cytology
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "Cytology" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "Cytology" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "Cytology" in x)]["Full Net LNM"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing Cytology: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# Survival
# What is the Average ROC for sets containing certain variables, with the range of ROC values
print("\n\n\n")
print("Survival")
# For CA125
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "CA125" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "CA125" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "CA125" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing CA125: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For MRI_MI
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "MRI_MI" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "MRI_MI" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "MRI_MI" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing MRI_MI: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For MSI
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "MSI" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "MSI" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "MSI" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing MSI: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For POLE
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "POLE" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "POLE" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "POLE" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing POLE: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For PreoperativeGrade
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "PreoperativeGrade" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "PreoperativeGrade" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "PreoperativeGrade" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing PreoperativeGrade: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For ER
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "ER" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "ER" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "ER" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing ER: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For PR
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "PR" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "PR" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "PR" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing PR: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For p53
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "p53" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "p53" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "p53" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing p53: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For L1CAM
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "L1CAM" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "L1CAM" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "L1CAM" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing L1CAM: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For Platelets
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "Platelets" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "Platelets" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "Platelets" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing Platelets: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)

# For Cytology
ROC  = ROC_results[ROC_results["Set"].apply(lambda x: "Cytology" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[0]))
min = ROC_results[ROC_results["Set"].apply(lambda x: "Cytology" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[1].split(",")[0][1:-1])).replace({0: np.nan}).min()
max = ROC_results[ROC_results["Set"].apply(lambda x: "Cytology" in x)]["Full Net Survival"].apply(lambda x: float(x.split(" ")[2].split(",")[0][0:-2])).max()

print("Average ROC for sets containing Cytology: ", round(ROC.mean(),2) , " with range: ", min, " - ", max)



